# Week 7: OUTER APPLY
## What is OUTER APPLY? 
`OUTER APPLY` is a means of first formulating subqueries, which are executed for each line in the result, and is then joined to the result. This is, of course, a means to be used with extreme caution, because since an `OUTER APPLY` is executed for every row in the result, even a cheap (sub-)query here can lead to high overall query costs.
## How can you use OUTER APPLY?
Imagine that you want to know in the Wide World Importers database for every item in stock when it was last ordered. If you would now join the items with the orders, you would have a huge result set, where you get every order for every item. From these you would then have to pick the last ones. Thereby you would work with a much larger amount of data than you actually want. This means that you would need more memory, more temp DB and more CPU time to process the query than the actual amount of data justifies. 
If you want to avoid this, you can use `OUTER APPLY` to execute a subquery for each line that completes this value. The whole thing will look like this:

In [0]:
SELECT 
     si.[StockItemID]
    ,si.[StockItemName]
    ,si.[UnitPrice] 
    ,od.[LastOrderDate] 
FROM [Warehouse].[StockItems] si
OUTER APPLY (
    SELECT TOP 1 
        o.[OrderDate] as LastOrderDate 
    FROM [Sales].[Orders] as o 
    LEFT JOIN [Sales].[OrderLines] as ol 
    ON o.[OrderID]= ol.[OrderID] 
    WHERE ol.StockItemID = si.StockItemID 
    ORDER BY o.[OrderDate] desc 
) od

If you now look at the execution plan for this query, you will see in the plan the subquery with 277 executions, as many as the query returns rows.

## #HERE'S ANOTHER IMAGE COMING #
This is of course not very efficient. In this example you can work with a CTE instead (you can read more about this in the first part of the series). The query would look like this:

In [0]:
;WITH cte_lastOrder as (
    SELECT 
         MAX(o.[OrderDate]) as LastOrderDate 
        ,ol.[StockItemID]
    FROM [Sales].[Orders] as o 
    LEFT JOIN [Sales].[OrderLines] as ol 
    ON o.[OrderID]= ol.[OrderID] 
    GROUP BY ol.StockItemID
)
SELECT 
     si.[StockItemID]
    ,si.[StockItemName]
    ,si.[UnitPrice] 
    ,lo.[LastOrderDate] 
FROM [Warehouse].[StockItems] si
LEFT JOIN cte_lastOrder lo 
ON si.StockItemID = lo.StockItemID

Now you may ask yourself: if this also works with CTEs, what do we need an `APPLY` operator for anyway? It's simple: you don't always want to merge tables, `APPLY` also gives you the possibility to execute a table valued function for each row of a query result and to join the result to the row, so you get the freedom to do many more operations than normal `JOIN` operations. 
By the way, an `OUTER APPLY` corresponds to a `LEFT JOIN`, while a `CROSS APPLY` corresponds to an `INNER JOIN` operation.
### References
- [Official Microsoft documentation](https://docs.microsoft.com/de-de/sql/t-sql/queries/from-transact-sql?view=sql-server-2017)